# **Combine Data**

## Joining Tables

In [4]:
%load_ext sql
%sql sqlite:///databases/world.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Inner Join

The `INNER JOIN` command looks for records that match in both tables, and only returns the matching records

<img src="images/inner_join.png" width="600"></img>

In [5]:
%%sql
SELECT pm.country, pm.continent, pm.prime_minister, p.president 
FROM prime_ministers AS pm
INNER JOIN presidents AS p USING(country)
LIMIT(5);

-- OR --

SELECT pm.country, pm.continent, pm.prime_minister, p.president 
FROM prime_ministers AS pm
INNER JOIN presidents AS p ON pm.country = p.country
LIMIT(5);

 * sqlite:///databases/world.db
Done.
Done.


country,continent,prime_minister,president
Egypt,Africa,Sherif Ismail,Abdel Fattah el-Sisi
Portugal,Europe,Antonio Costa,Marcelo Rebelo de Sousa
Vietnam,Asia,Nguyen Xuan Phuc,Tran Dai Quang
Haiti,North America,Jack Guy Lafontant,Jovenel Moise


One can join on multiple keys with the `AND` keyword

<img src="images/inner_join_and.png" width="800"></img>


You can join multiple tables by chaining `JOIN ON` commands

<img src="images/inner_join_chain.png" width="700"></img>


In [6]:
%%sql
SELECT 
    ci.name AS city_name, 
    co.name AS country_name, 
    cu.basic_unit AS currency
FROM cities AS ci
INNER JOIN countries AS co ON co.code = ci.country_code
INNER JOIN currencies AS cu ON cu.code = ci.country_code
LIMIT(5)

 * sqlite:///databases/world.db
Done.


city_name,country_name,currency
Abidjan,Cote d'Ivoire,West African CFA franc
Abu Dhabi,United Arab Emirates,United Arab Emirates dirham
Abuja,Nigeria,Nigerian naira
Accra,Ghana,Ghanaian cedi
Addis Ababa,Ethiopia,Ethiopian birr


### Outer Join 

#### Left & Right Outer Join


There are two variants of the outer join command, namely `LEFT OUTER JOIN` and `RIGHT OUTER JOIN`. These commands will look  for records that match in both tables, and return all the records from the left or right table. If the table that is being joined doesn’t contain values for a certain record, then a null value is returned.

`LEFT OUTER JOIN` and `RIGHT OUTER JOIN` are equivalent; a left outer join can be rewritten into a right outer join based on the syntax of the SQL command. 

When performing a left or right join, the `OUTER` command can be omitted. `LEFT OUTER JOIN` is equivalent to `LEFT JOIN`

<img src="images/left_outer_join.png" width="600"></img>


In [7]:
%%sql 
SELECT pm.country, pm.prime_minister, pr.president
FROM prime_ministers AS pm
LEFT JOIN presidents AS pr ON pm.country = pr.country
LIMIT(5)

 * sqlite:///databases/world.db
Done.


country,prime_minister,president
Egypt,Sherif Ismail,Abdel Fattah el-Sisi
Portugal,Antonio Costa,Marcelo Rebelo de Sousa
Vietnam,Nguyen Xuan Phuc,Tran Dai Quang
Haiti,Jack Guy Lafontant,Jovenel Moise
India,Narendra Modi,None


#### Full Outer Join

The `FULL JOIN `or `FULL OUTER JOIN` command returns all values in both tables, if there is no match a `null` value will be returned

<img src="images/full_outer_join.png" width="600"></img>


In [8]:
%%sql 
SELECT pm.country, pm.prime_minister, pr.president
FROM prime_ministers AS pm
FULL JOIN presidents AS pr ON pm.country = pr.country
LIMIT(5)

 * sqlite:///databases/world.db
Done.


country,prime_minister,president
Egypt,Sherif Ismail,Abdel Fattah el-Sisi
Portugal,Antonio Costa,Marcelo Rebelo de Sousa
Vietnam,Nguyen Xuan Phuc,Tran Dai Quang
Haiti,Jack Guy Lafontant,Jovenel Moise
India,Narendra Modi,None


### Cross Join

The `CROSS JOIN` command returns all possible combinations of the records of two tables. 

<img src="images/cross_join.png" width="300"></img>


In [9]:
%%sql 
SELECT pm.country, pm.prime_minister, pr.president
FROM prime_ministers AS pm
CROSS JOIN presidents AS pr
LIMIT(5)

 * sqlite:///databases/world.db
Done.


country,prime_minister,president
Egypt,Sherif Ismail,Abdel Fattah el-Sisi
Egypt,Sherif Ismail,Marcelo Rebelo de Sousa
Egypt,Sherif Ismail,Jovenel Moise
Egypt,Sherif Ismail,Jose Mujica
Egypt,Sherif Ismail,Ellen Johnson Sirleaf


### Self Join

It is also possible to join a table with itself in order to compare the records of a single table with each other. There is no dedicated syntax for this operation, instead an `INNER JOIN` is performed using the same table.  

In [10]:
%%sql
SELECT 
    pr1.country AS country1, 
    pr2.country AS country2, 
    pr1.continent AS continent
FROM prime_ministers AS pr1
INNER JOIN prime_ministers AS pr2 
    ON pr1.continent = pr2.continent -- compare countries from same continents
    AND pr1.country <> pr2.country -- don't compare country with itself
LIMIT(5)


 * sqlite:///databases/world.db
Done.


country1,country2,continent
Portugal,Norway,Europe
Portugal,Spain,Europe
Vietnam,Brunei,Asia
Vietnam,India,Asia
Vietnam,Oman,Asia


## Set Operations


Another way of combining tables is through set operations. From set theory there are different ways of visualizing sets using venn diagrams. 

<img src="images/set_operations.png" width="500"></img>


### Union & Union All

Union operations will only work if the two tables have
- The same number of columns
- Same data types in the columns

The `UNION` operation takes two tables and returns all records from both tables. If two records from the two tables are identical the duplicates are removed. 

<img src="images/union.png" width="400"></img>




In [14]:
%%sql 
SELECT monarch AS leader, country
FROM monarchs
UNION 
    SELECT prime_minister, country 
    FROM prime_ministers
ORDER BY country, leader
LIMIT(5)

 * sqlite:///databases/world.db
Done.


leader,country
Malcolm Turnbull,Australia
Hassanal Bolkiah,Brunei
Sherif Ismail,Egypt
Jack Guy Lafontant,Haiti
Narendra Modi,India


The `UNION ALL` operation is similar to the `UNION` operation, except the duplicates are not removed

<img src="images/union_all.png" width="400"></img>


In [15]:
%%sql 
SELECT monarch AS leader, country
FROM monarchs
UNION ALL
    SELECT prime_minister, country 
    FROM prime_ministers
ORDER BY country, leader
LIMIT(5)

 * sqlite:///databases/world.db
Done.


leader,country
Malcolm Turnbull,Australia
Hassanal Bolkiah,Brunei
Hassanal Bolkiah,Brunei
Sherif Ismail,Egypt
Jack Guy Lafontant,Haiti


### Intersect

The `INTERSECT` operation only returns the records which are identical (shared) in the two tables. This is similar to an `INNER JOIN` operation with an `AND` clause, the difference is that the `INTERSECT` operations only returns unique rows.

<img src="images/intersect_vs_inner_join.png" width="800"></img>



In [17]:
%%sql 
SELECT monarch AS leader, country
FROM monarchs
INTERSECT 
    SELECT prime_minister, country 
    FROM prime_ministers
LIMIT(5)

 * sqlite:///databases/world.db
Done.


leader,country
Hassanal Bolkiah,Brunei
Qaboos bin Said al Said,Oman


### Except

The `EXCEPT` operation identifies the records that are present in one table, but not in the other. The entire record/row/entry needs to match (all column values). 


<img src="images/except.png" width="500"></img>


In [18]:
%%sql 
SELECT monarch AS leader, country
FROM monarchs
EXCEPT 
    SELECT prime_minister, country 
    FROM prime_ministers
LIMIT(5)

 * sqlite:///databases/world.db
Done.


leader,country
Felipe VI,Spain
Harald V,Norway
